Link to dataset including description: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7

In [8]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf()
conf.setAppName("ML Data Preparation")
conf.setMaster("local")
conf.set("spark.hadoop.fs.defaultFS", "file:///")
sc = SparkContext.getOrCreate(conf)
sc.setLogLevel("ERROR")
spark = SparkSession.builder.appName("App") \
    .getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [27]:
dataframe = spark.read.load("./data/1987.csv.bz2", format="csv", sep=",", inferSchema="true", header="true")

# Drop the features for which the values are not available at the time of departure
dataframe = dataframe.drop(
    "ArrTime", 
    "ActualElapsedTime", 
    "AirTime", 
    "TaxiIn", 
    "Diverted", 
    "CarrierDelay", 
    "WeatherDelay", 
    "NASDelay", 
    "SecurityDelay", 
    "LateAircraftDelay"
)

In [28]:
print("Number of elements:", dataframe.count())

Number of elements: 1311826


We will try to predict `ArrDelay` with the help of the other variables.

1. `Year` 1987-2008
2. `Month` 1-12
3. `DayofMonth` 1-31
4. `DayOfWeek` 1 (Monday) - 7 (Sunday)
5. `DepTime` actual departure time (local, hhm m)
6. `CRSDepTime` scheduled departure time (local, hhmm)
7. `ArrTime` actual arrival time (local, hhmm)
8. `CRSArrTime` scheduled arrival time (local, hhmm)
9. `UniqueCarrier` unique carrier code
10. `FlightNum` flight number
11. `TailNum` plane tail number
12. `ActualElapsedTime` (in minutes)
13. `CRSElapsedTime` (in minutes)
14. `AirTime` (in minutes)
15. `ArrDelay` arrival delay, in minutes
16. `DepDelay` departure delay, in minutes
17. `Origin` origin IATA airport code
18. `Dest` des tination IATA airport code
19. `Distance` in miles
20. `TaxiIn` taxi in time, in minutes
21. `TaxiOut` taxi out time in minutes
22. `Cancelled` was the flight cancelled?
23. `CancellationCode` reason for cancellation (A = carrier, B = weather, C = NAS, D = security)
24. `Diverted` 1 = yes, 0 = no
25. `CarrierDelay` in minutes
26. `WeatherDelay` in minutes
27. `NASDelay` in minutes
28. `SecurityDelay` in minutes
29. `LateAircraftDelay` in minutes

In [29]:
dataframe.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)



In [30]:
dataframe.show()

+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|Cancelled|CancellationCode|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|1987|   10|        14|        3|    741|       730|       849|           PS|     1451|     NA|            79|      23|      11|   SAN| SFO|     447|     NA|        0|              NA|
|1987|   10|        15|        4|    729|       730|       849|           PS|     1451|     NA|            79|      14|      -1|   SAN| SFO|     447|     NA|        0|              NA|
|1987|   10|        17|        6|    741|       730|       849|           P

Flights which were cancelled (`Cancelled = 1`) do not have an arrival delay (`ArrDelay = NA`).
$\newline\Rightarrow$ Filter out these rows before applying the model.

In [37]:
dataframe.where(dataframe.Cancelled == 1).select("Cancelled", "ArrDelay").show(10)

+---------+--------+
|Cancelled|ArrDelay|
+---------+--------+
|        1|      NA|
|        1|      NA|
|        1|      NA|
|        1|      NA|
|        1|      NA|
|        1|      NA|
|        1|      NA|
|        1|      NA|
|        1|      NA|
|        1|      NA|
+---------+--------+
only showing top 10 rows

